# Extraction of memory usage time series
⚠️ **The VPN must be active!**

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl
from prometheus_api_client import PrometheusConnect
from prometheus_api_client.utils import parse_datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np

## Extraction
- Query data from Prometheus
- Store a dump of data to the local FS
### Parameters
`k8s_cluster`: Kubernetes cluster's identifier where the Prometheus server runs
- `"prod"`
- `"preprod"`

`workload`: Name of a Kubernetes _statefulset_ or _deployment_ in the selected Kubernetes cluster

In [ ]:
# Parameters
k8s_cluster="preprod"
workload="nlp-data"
range_from=60 # days
range_to=0 # days, 0 = "now"

from prom_usage_service import PromUsageService
prom_service = PromUsageService(cluster=k8s_cluster, workload=workload)
time_series = prom_service.query(start=range_from, end=range_to, metric=PromUsageService.Metrics.MEMORY_MIB)
prom_service.save_cache("./data/mem_usage.json", overwrite=True)

In [ ]:
time_series.resource

## Deserialization / raw data plot

In [ ]:
from stats.stats_service import StatsService
stats_service = StatsService(name="Memory usage")
stats_service.load_time_series(path="data/mem_usage.json")

In [ ]:
fig, ax = plt.subplots()
ax.plot(stats_service.time_series.time, stats_service.time_series.resource, linewidth=1.0)
plt.xticks(rotation = 45)
plt.show()